In [308]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import re
# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [309]:
path = 'C:/Users/user/Documents/mydocs/rs_geek_brains/data/'

In [310]:
data = pd.read_csv(path + 'retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [311]:
#на тест берет 3 последниенедели

test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [312]:
data_train.shape[0], data_test.shape[0]

(2278490, 118314)

In [313]:
data_train.item_id.nunique()

86865

# 1. Baseline

Создадим датафрейм с покупками юзеров на тестовом датасете (последние 3 недели)

In [314]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


### 1.1 Random recommendation

In [315]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [316]:
%%time

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
result.head(2)

Wall time: 3.13 s


,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[921259, 2261348, 13115883, 5566687, 68748]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[651824, 900112, 15571301, 958676, 991230]"


### 1.2 Popularity-based recommendation

In [317]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    recs = popular.head(n).item_id
    return recs.tolist()

In [318]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

Wall time: 113 ms


,user_id,actual,random_recommendation,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[921259, 2261348, 13115883, 5566687, 68748]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[651824, 900112, 15571301, 958676, 991230]","[6534178, 6533889, 1029743, 6534166, 1082185]"


### 1.3 Weighted random recommender - hometask 

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [ ]:
## Создадим item profile на основе суммы продаж по каждому item и посчитаем log от этой суммы, чтобы использовать, как список вероятности

In [319]:
item_profile = data_train.groupby('item_id')['sales_value'].sum().reset_index()

In [320]:
item_profile['log_sales'] = np.log(item_profile.sales_value)

C:\Users\user\anaconda3\lib\site-packages\pandas\core\series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [321]:
#где не было продаж поставим 0
item_profile.loc[item_profile.sales_value==0.0, 'log_sales'] = 0

#где отрицательные значения поставим 0
item_profile.loc[item_profile.log_sales < 0, 'log_sales'] = 0

#поделим все значения на сумму логарифмов, чтобы в итоге была 1, как общая вероятность 
item_profile['log_p'] = item_profile.log_sales/ item_profile.log_sales.sum()

In [322]:
item_profile.head()

,item_id,sales_value,log_sales,log_p
0,25671,20.94,3.041661,0.000013
1,26081,0.99,0.000000,0.000000
2,26093,1.59,0.463734,0.000002
3,26190,1.54,0.431782,0.000002
4,26355,1.98,0.683097,0.000003


In [323]:
def weighted_random_recommendation (items, sales_log,  n=5):
    """Случайные рекоммендации"""
    items = np.array(items)
    probability = np.array(sales_log) #вероятность рекомендации
    recs = np.random.choice(items, size=n, replace=False, p = probability)
    return recs.tolist()

In [324]:
items_train = item_profile.item_id
prob_sales = item_profile.log_p

In [325]:
%%time
result['w_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_train, prob_sales, n=5))
result.head(2)

Wall time: 1.57 s


,user_id,actual,random_recommendation,popular_recommendation,w_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[921259, 2261348, 13115883, 5566687, 68748]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1084342, 1127003, 6391541, 1138296, 9527256]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[651824, 900112, 15571301, 958676, 991230]","[6534178, 6533889, 1029743, 6534166, 1082185]","[13381780, 9673402, 9554210, 1022795, 1135253]"


## 2. Item-Item Recommender / ItemKNN

In [326]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [327]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [328]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

C:\Users\user\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [329]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [330]:
%%time

model = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=True, 
                        filter_items=None, 
                        recalculate_user=True)


Wall time: 626 ms


In [331]:
%%time

result['itemitem'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=True, 
                                    filter_items=None, 
                                    recalculate_user=True)])

Wall time: 49.9 ms


In [332]:
result.head()

,user_id,actual,random_recommendation,popular_recommendation,w_random_recommendation,itemitem
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[921259, 2261348, 13115883, 5566687, 68748]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1084342, 1127003, 6391541, 1138296, 9527256]","[981760, 1127831, 1098066, 826249, 878996]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[651824, 900112, 15571301, 958676, 991230]","[6534178, 6533889, 1029743, 6534166, 1082185]","[13381780, 9673402, 9554210, 1022795, 1135253]","[981760, 995242, 1029743, 840361, 961554]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[12604934, 952127, 7407873, 6704164, 2922387]","[6534178, 6533889, 1029743, 6534166, 1082185]","[972743, 991012, 1108812, 10285022, 9553031]","[1098066, 826249, 1106523, 923746, 1058997]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1003201, 5995225, 1110160, 15571500, 9484494]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1791199, 10284834, 12670116, 5996184, 15830847]","[981760, 1098066, 840361, 883404, 916122]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1853330, 838946, 7410249, 5586131, 9245489]","[6534178, 6533889, 1029743, 6534166, 1082185]","[15973176, 13776765, 889585, 955653, 1087850]","[904360, 13115903, 13189726, 13190294, 15596515]"


## 3. Косинусное сходство и CosineRecommender

In [333]:
%%time

model = CosineRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=True, 
                        filter_items=None, 
                        recalculate_user=False)


Wall time: 625 ms


In [334]:
%%time

result['cosine'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=True, 
                                    filter_items=None, 
                                    recalculate_user=True)])

Wall time: 68.8 ms


In [335]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,w_random_recommendation,itemitem,cosine
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[921259, 2261348, 13115883, 5566687, 68748]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1084342, 1127003, 6391541, 1138296, 9527256]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[651824, 900112, 15571301, 958676, 991230]","[6534178, 6533889, 1029743, 6534166, 1082185]","[13381780, 9673402, 9554210, 1022795, 1135253]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]"


## 4. TF-IDF взвешивание и TFIDFRecommender

In [336]:
%%time

model = TFIDFRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=True, 
                        filter_items=None, 
                        recalculate_user=False)


Wall time: 669 ms


In [337]:
%%time

result['tfidf'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=True, 
                                    filter_items=None, 
                                    recalculate_user=False)])

Wall time: 65.8 ms


In [338]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,w_random_recommendation,itemitem,cosine,tfidf
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[921259, 2261348, 13115883, 5566687, 68748]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1084342, 1127003, 6391541, 1138296, 9527256]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[651824, 900112, 15571301, 958676, 991230]","[6534178, 6533889, 1029743, 6534166, 1082185]","[13381780, 9673402, 9554210, 1022795, 1135253]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]"


## 5. Own purchases

In [339]:
%%time

model = ItemItemRecommender(K=1, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=True, 
                        filter_items=None, 
                        recalculate_user=False)


Wall time: 630 ms


In [340]:
%%time

result['own_purchases'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[999999], 
                                    recalculate_user=False)])

Wall time: 42.9 ms


In [341]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,w_random_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[921259, 2261348, 13115883, 5566687, 68748]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1084342, 1127003, 6391541, 1138296, 9527256]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[651824, 900112, 15571301, 958676, 991230]","[6534178, 6533889, 1029743, 6534166, 1082185]","[13381780, 9673402, 9554210, 1022795, 1135253]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


### Задание 2. Расчет метрик

In [343]:
def precision_at_k(recommended_list, bought_list, k = 5 ):
    
    #recommended_list = [re.sub(r'\W', '', x) for x in recommended_list.split()]
    #bought_list = [re.sub(r'\W', '', x) for x in bought_list.split()][:k]
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    bought_list = bought_list  
    recommended_list = recommended_list[:k]
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)
    return precision

In [350]:
rr = result.apply(lambda x: precision_at_k(x['random_recommendation'],  x['actual'], k = 5), axis  =1).mean()
pr = result.apply(lambda x: precision_at_k(x['popular_recommendation'],  x['actual'], k = 5), axis  =1).mean()
wr = result.apply(lambda x: precision_at_k(x['w_random_recommendation'],  x['actual'], k = 5), axis  =1).mean()
ii = result.apply(lambda x: precision_at_k(x['itemitem'],  x['actual'], k = 5), axis  =1).mean()
cos = result.apply(lambda x: precision_at_k(x['cosine'],  x['actual'], k = 5), axis  =1).mean()
tf = result.apply(lambda x: precision_at_k(x['tfidf'],  x['actual'], k = 5), axis  =1).mean()
own = result.apply(lambda x: precision_at_k(x['own_purchases'],  x['actual'], k = 5), axis  =1).mean()

In [352]:
res_d = {'random_recommendation' : rr, 'popular_recommendation': pr, 'w_random_recommendation': wr, 'itemitem': ii, 'cosine': cos, 'tfidf': tf,'own_purchases': own}

In [361]:
df_res = pd.DataFrame.from_dict(res_d, orient='index', columns = ['result'])
df_res.sort_values(by = 'result')

,result
random_recommendation,0.000686
w_random_recommendation,0.001273
itemitem,0.033595
cosine,0.035260
tfidf,0.036141
popular_recommendation,0.155240
own_purchases,0.179693


### Резюме
    
Лучший результат - это модель, где ориентируемся на собственные покупки пользователя. Интересно, что взвешенная рандом модель по логарифму работает лучше, чем обычная. На практике, скорее всего зависит еще от степени новизны рекомендаций. 

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ- {2000, 5000, 10000} товаров. Постройте график зависимости precision@5 от топ-*
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$. Постройте график зависимости precision@5 от $K$ 